In [1]:
import xarray as xr
import os
import tensorflow as tf
import jdutil # ./jdutil.py
from util import *  # ./util.py
import datetime
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
from sklearn.model_selection import train_test_split

print('TensorFlow version:',tf.__version__)

TensorFlow version: 2.10.0


In [25]:
atms_npp = xr.load_dataset('.\\data\\atms_npp_surface_contribution.nc')
atms_npp = atms_npp.drop_sel(channel=[8,9,10,11,12,13,14,15,19,20,21,22]).dropna(dim="obs_id",how="any",subset=['surface_contribution'])
atms_npp = atms_npp.where(atms_npp.lsm == 1.0, drop=True)
atms_npp = atms_npp.where(atms_npp.tcc <= 0.1, drop=True)
atms_npp

<xarray.Dataset>
Dimensions:               (level: 137, obs_id: 1193, channel: 10)
Coordinates:
  * level                 (level) int64 1 2 3 4 5 6 ... 132 133 134 135 136 137
    jd2                   float64 2.458e+06
  * channel               (channel) int64 1 2 3 4 5 6 7 16 17 18
    lat2                  (obs_id) float64 68.4 68.4 68.4 ... 23.4 23.1 23.1
    lon2                  (obs_id) float64 -166.2 -165.9 ... -109.8 -106.2
Dimensions without coordinates: obs_id
Data variables: (12/151)
    crwc                  (level, obs_id) float64 -5.821e-11 ... -5.821e-11
    cswc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    t                     (level, obs_id) float64 170.4 170.3 ... 302.6 305.4
    q                     (level, obs_id) float64 2.429e-06 ... 0.01498
    o3                    (level, obs_id) float64 1.131e-07 ... 5.596e-08
    clwc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                    ...
    scan_line_stddev      (obs_id) float64 0.7071 nan nan ... nan 0.7071 0.7071
    scan_line_count       (obs_id) float64 2.0 1.0 1.0 2.0 ... 1.0 1.0 2.0 2.0
    fov1                  (obs_id) float64 9.0 9.0 10.0 9.0 ... 94.0 86.0 94.0
    fov1_stddev           (obs_id) float64 0.0 nan nan ... nan 4.767e-06 0.0
    fov1_count            (obs_id) float64 2.0 1.0 1.0 2.0 ... 1.0 1.0 2.0 2.0
    surface_contribution  (obs_id, channel) float64 0.6102 0.6098 ... 0.6705

In [26]:
atms_n20 = xr.load_dataset('.\\data\\atms_n20_surface_contribution.nc')
atms_n20 = atms_n20.drop_sel(channel=[8,9,10,11,12,13,14,15,19,20,21,22]).dropna(dim="obs_id",how="any",subset=['surface_contribution'])
atms_n20 = atms_n20.where(atms_n20.lsm == 1.0, drop=True)
atms_n20 = atms_n20.where(atms_n20.tcc <= 0.1, drop=True)
atms_n20

<xarray.Dataset>
Dimensions:               (level: 137, obs_id: 7733, channel: 10)
Coordinates:
  * level                 (level) int64 1 2 3 4 5 6 ... 132 133 134 135 136 137
    jd2                   float64 2.458e+06
  * channel               (channel) int64 1 2 3 4 5 6 7 16 17 18
    lat2                  (obs_id) float64 -6.0 -6.0 -6.0 ... -87.0 -87.0 -87.0
    lon2                  (obs_id) float64 -68.1 -67.8 -67.5 ... -91.2 -90.9
Dimensions without coordinates: obs_id
Data variables: (12/151)
    crwc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    cswc                  (level, obs_id) float64 -2.328e-10 ... -2.328e-10
    t                     (level, obs_id) float64 188.4 188.3 ... 227.7 227.8
    q                     (level, obs_id) float64 2.018e-06 ... 5.326e-05
    o3                    (level, obs_id) float64 1.757e-07 ... 6.389e-08
    clwc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                    ...
    scan_line_stddev      (obs_id) float64 0.5523 1.502e-05 ... nan nan
    scan_line_count       (obs_id) float64 3.0 2.0 2.0 1.0 ... 2.0 2.0 1.0 1.0
    fov1                  (obs_id) float64 67.3 65.5 63.57 ... 10.0 10.0 10.0
    fov1_stddev           (obs_id) float64 0.5653 0.7071 0.7071 ... nan nan
    fov1_count            (obs_id) float64 3.0 2.0 2.0 1.0 ... 2.0 2.0 1.0 1.0
    surface_contribution  (obs_id, channel) float64 0.9337 0.9327 ... 0.8243

In [56]:
variables = [
        'lat1',
        'lon1',
        'zenith_angle',
        'azimuth_angle',
        'skt',
        'u10n',
        'v10n',
        'u10',
        'v10',
        'sp',
        'msl',
        'fsr',
        'u100',
        'v100',
        'cvl',
        'cvh',
        'tvl',
        'tvh',
        'swvl1',
        'swvl2',
        'swvl3',
        'swvl4',
        'slt',
        'lai_lv',
        'lai_hv',
        'sdfor',
        'stl1',
        'stl2',
        'stl3',
        'stl4',
        'd2m'
    ]

x_train = atms_n20[variables].to_array().values.T
x_test  = atms_npp[variables].to_array().values.T
y_train = atms_n20['surface_contribution'].values
y_test  = atms_npp['surface_contribution'].values

print('max y_train', max(y_train.flatten()))
print('min y_train', min(y_train.flatten()))
print('max y_test', max(y_test.flatten()))
print('min y_test', min(y_test.flatten()))

x_train_mean = x_train.mean(axis=0)
x_train_std  = x_train.std(axis=0)
y_train_mean = y_train.mean(axis=0)

x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean

max y_train 2.840763135669331
min y_train -2.48286935586506
max y_test 1.075257588053412
min y_test -6.207817292420018


In [57]:
# train normally

model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(x_train.shape[1]),
            tf.keras.layers.Dense(50, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(50, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(50, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(10)
        ])

loss_fn = tf.keras.losses.MeanAbsoluteError()

model.compile(optimizer='adam',
              loss=loss_fn)

hist = model.fit(x_train, y_train, 
          epochs=50,
          validation_data=(x_test, y_test)
       )

Epoch 1/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0831 - val_loss: 0.0993
Epoch 2/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0425 - val_loss: 0.0904
Epoch 3/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0363 - val_loss: 0.0712
Epoch 4/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0334 - val_loss: 0.0708
Epoch 5/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0313 - val_loss: 0.0689
Epoch 6/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0300 - val_loss: 0.0606
Epoch 7/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0290 - val_loss: 0.0673
Epoch 8/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0281 - val_loss: 0.0608
Epoch 9/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0274 - val_loss: 0.0649
Epoch 10/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0270 - val_loss: 0.0581

In [70]:
model.save("surface_contribution_model_3hidden50.h5")
new_model = tf.keras.models.load_model("surface_contribution_model_3hidden50.h5")
new_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_61 (Dense)            (None, 31)                992       
                                                                 
 dense_62 (Dense)            (None, 50)                1600      
                                                                 
 dropout_6 (Dropout)         (None, 50)                0         
                                                                 
 dense_63 (Dense)            (None, 50)                2550      
                                                                 
 dropout_7 (Dropout)         (None, 50)                0         
                                                                 
 dense_64 (Dense)            (None, 50)                2550      
                                                                 
 dropout_8 (Dropout)         (None, 50)              

In [66]:
loss = model.evaluate(x_test, y_test)
print('loss',loss)
y_pred = model.predict(x_test)
print('average error in each variable',abs(y_test - y_pred).mean())
y_pred_unnormalized = y_pred + y_train_mean

ds = xr.Dataset(
    data_vars=dict(prediction=(["obs_id", "channel"], y_pred_unnormalized))
)
ds.to_netcdf("atms_npp_surface_contribution_predictions.nc")

38/38 [==============================] - 0s 1ms/step - loss: 0.0545
loss 0.054466504603624344
38/38 [==============================] - 0s 1ms/step
average error in each variable 0.05446649926570644


In [43]:
# test model from loaded weights

current_variables = [4, 10 ,15, 27]

x_train_subset = x_train[:,current_variables]
x_test_subset = x_test[:,current_variables]

model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(x_train_subset.shape[1]),
            tf.keras.layers.Dense(30, activation='relu'),
            tf.keras.layers.Dense(22)
        ])

loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(optimizer='adam',
              loss=loss_fn)
model.build(input_shape=(None,4))
model.load_weights('.\\checkpoints\\shallow_h30[4, 10, 15, 27].h5')
loss = model.evaluate(x_test_subset, y_test)
print('loss',loss)
y_pred = model.predict(x_test_subset)
print('average error in each variable',abs(y_test - y_pred).mean())
y_pred_unnormalized = y_pred + y_train_mean
y_pred_unnormalized

38/38 [==============================] - 0s 1ms/step - loss: 74.2951
loss 74.29512023925781
38/38 [==============================] - 0s 1ms/step
average error in each variable 6.28922325935255


array([[268.49682017, 266.01462367, 270.07175542, ..., 265.41046595,
        259.08116687, 253.39722832],
       [266.96065303, 264.59486583, 268.91334249, ..., 264.82364726,
        258.67306674, 253.15303334],
       [265.53070613, 263.27310374, 267.83790494, ..., 264.27115893,
        258.28557552, 252.91893872],
       ...,
       [287.94321605, 284.22502711, 284.87660695, ..., 273.81030154,
        265.33366359, 257.59867486],
       [286.14388438, 282.74547389, 282.98639394, ..., 275.33169627,
        267.09293521, 259.42340668],
       [288.6281335 , 284.89311793, 285.34534551, ..., 274.4826076 ,
        265.8982159 , 258.07776555]])

In [48]:
ds = xr.Dataset(
    data_vars=dict(prediction=(["obs_id", "channel"], y_pred_unnormalized))
)
ds.to_netcdf("atms_npp_predictions.nc")

In [79]:
atms_n20['surface_contribution'].dropna(dim="channel", how="all")

<xarray.DataArray 'surface_contribution' (obs_id: 7733, channel: 16)>
array([[0.93369412, 0.93265616, 0.91953647, ...,        nan,        nan,
               nan],
       [0.93657966, 0.93527977, 0.92414386, ...,        nan,        nan,
               nan],
       [0.94384561, 0.94215341, 0.93825967, ...,        nan,        nan,
               nan],
       ...,
       [0.82475992, 0.82104946, 0.79174466, ..., 0.73642687, 0.6665216 ,
        0.58923724],
       [0.82298695, 0.81854393, 0.78744012, ..., 0.72574574, 0.65496386,
        0.57781247],
       [0.82280915, 0.81837434, 0.78727378, ..., 0.72483403, 0.6536964 ,
        0.57628052]])
Coordinates:
    jd2      float64 2.458e+06
  * channel  (channel) int64 1 2 3 4 5 6 7 8 9 16 17 18 19 20 21 22
    lat2     (obs_id) float64 -6.0 -6.0 -6.0 -6.0 ... -87.0 -87.0 -87.0 -87.0
    lon2     (obs_id) float64 -68.1 -67.8 -67.5 -67.2 ... -91.5 -91.2 -90.9
Dimensions without coordinates: obs_id